https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import os
from glob import glob

In [2]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
os.listdir("../dataset/mnist_png/training")

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [4]:
train_dir = '../dataset/mnist_png/training'
test_dir = '../dataset/mnist_png/testing'

## Hyperparameter Tunning

In [5]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (28, 28, 1)
num_classes = 10

## Preprocess

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

- 만약 flow_from_directory가 class 개수를 잘못 찾으면 classes 파라미터로 변경가능!

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2], # channel 제외
    batch_size=batch_size,
    color_mode='grayscale'
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale'
)

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [8]:
len(train_generator), len(validation_generator)

(1875, 313)

## Build Model

In [9]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [11]:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

C:\Users\jumin\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1875/1875 [==============================] - 261s 138ms/step - loss: 0.9185 - accuracy: 0.6837 - val_loss: 0.5321 - val_accuracy: 0.8418
Epoch 2/10
1875/1875 [==============================] - 216s 115ms/step - loss: 0.6287 - accuracy: 0.7871 - val_loss: 0.5040 - val_accuracy: 0.8543
Epoch 3/10
1875/1875 [==============================] - 215s 115ms/step - loss: 0.5144 - accuracy: 0.8281 - val_loss: 0.3174 - val_accuracy: 0.9033
Epoch 4/10
1875/1875 [==============================] - 224s 119ms/step - loss: 0.4371 - accuracy: 0.8594 - val_loss: 0.1766 - val_accuracy: 0.9501
Epoch 5/10
1875/1875 [==============================] - 221s 118ms/step - loss: 0.3825 - accuracy: 0.8785 - val_loss: 0.1777 - val_accuracy: 0.9479
Epoch 6/10
1875/1875 [==============================] - 223s 119ms/step - loss: 0.3531 - accuracy: 0.8890 - val_loss: 0.1429 - val_accuracy: 0.9570
Epoch 7/10
1875/1875 [==============================] - 221s 118ms/step - loss: 0.3309 - accuracy: 0.8974 - val_